Fist cleaning the dataset is a must.
the dataset is orginized like so:
dev-articles -> Articles for testing the results
train-articles -> Articles for training the model
train-labels-task1 -> Labels of propaganda technique in articles
train-labels-task2 -> Labels of propaganda technique with lines in articles

Firstly, it is needed to gather labels from the task1 folder, put them in a dict where the article number is the key and the start and end of the propaganda techniques are values.

In [39]:
import os

# Access the articles and sort them
articles = os.listdir("datasets/train-articles/")
articles.sort()

# Access the labels and sort them
propagandaLabels = os.listdir("datasets/train-labels-task1-span-identification/")
propagandaLabels.sort()

# Create the dictonary for the article number and techniques
propLabels = {}

for i in range(len(articles)):
    article = articles[i]
    article = os.path.splitext(article)[0] # Remove the .txt in file name
    articles[i] = article
    articleNumber = article[7:]
    
    labelPath = "datasets/train-labels-task1-span-identification/{articleName}.task1-SI.labels".format(articleName=article)
    with open(labelPath) as f:
        labels = f.readlines()
        
        # replace \t and \n in tags with " " for easier processing later on
        for i in range(len(labels)):
            label = labels[i]
            label = label.replace("\t", " ")
            label = label.replace("\n", " ")
            labels[i] = label
        propLabels[article] = labels
    f.close()
    
print(propLabels[articles[0]])

['111111111 265 323 ', '111111111 1795 1935 ', '111111111 149 157 ', '111111111 1069 1091 ', '111111111 1334 1462 ', '111111111 1577 1616 ', '111111111 2023 2086 ']


Now what I did to taks 1 will now need to occur to task 2

In [40]:
propagandaLabelsTechnique = os.listdir("datasets/train-labels-task2-technique-classification") # this is where our tags are located for techique
## note: in span-identification, techniques that overlap are not split
propagandaLabelsTechnique.sort()
propLabelsTechnique = {} # Dictionary containing news article as key, and technique as value

for i in range(len(articles)):
    article = articles[i]
    articleNoExt = os.path.splitext(article)[0] # remove the .txt file extension ([2])
    articles[i] = articleNoExt # replace articles[i] with the same name but without the .txt extension
    articleNumber = articleNoExt.replace('article', '') # remove 'article' to leave just the number
    
    labelsPath = "datasets/train-labels-task2-technique-classification/"+ articleNoExt + ".task2-TC.labels"
    
    with open(labelsPath) as f:
        labels = f.readlines()
        # replace \t and \n in tags with " " for easier processing later on
        for i in range(len(labels)):
            label = labels[i]
            label = label.replace(articleNumber, " ")
            label = label.replace("\t", " ")
            label = label.replace("\n", " ")
            labels[i] = label 
            #print(tag)
        propLabelsTechnique[articleNoExt] = labels
    f.close()
    
print(propLabelsTechnique[articles[0]])

['  Appeal_to_Authority 265 323 ', '  Appeal_to_Authority 1795 1935 ', '  Doubt 149 157 ', '  Repetition 1069 1091 ', '  Appeal_to_fear-prejudice 1334 1462 ', '  Appeal_to_fear-prejudice 1577 1616 ', '  Appeal_to_fear-prejudice 1856 1910 ', '  Appeal_to_fear-prejudice 2023 2086 ']


Using the dict created, read all of the sentences that have been annotated as "propaganda" from the 'train-articles' folder, and put them in a list which will be named 'propSentencesSpan'.

In [ ]:
propSentencesSpan = []

for article in articles:
    artPath = "datasets/train-articles/" + article + ".txt"
    
    labels = propLabels[article]
    
    with open(artPath, encoding="utf-8") as f:
        wholeArticle = f.read()
        for label in labels:
            label = label.split()
            start = int(label[1])
            end = int(label[2])
            
            labeledLine = wholeArticle[start:end]
            labeledLine = labeledLine.replace("\n", " ")
            labeledLine = labeledLine.replace("\t", " ")
          
            propSentencesSpan.append(labeledLine)
    f.close()
    
print(propSentencesSpan[0])

Create a dictionary with the keys being the propoganda sentences, and the values being their associated propoganda type. This is to setup the data to be put into a Pandas dataframe.

In [ ]:
# dictionary with sentences as keys and technique as value
propagandaTechniques = {}
propagandaTechniques["Sentence"] = []
propagandaTechniques["Technique"] = []

for article in articles:
    artPath = "datasets/train-articles/" + article + ".txt"
    
    labels = propLabelsTechnique[article]
    with open(artPath, encoding="utf-8") as f:
        wholeArticle = f.read()
        for label in labels:
            label = label.split()
            propagandaTechniques["Technique"].append(label[0]) # add technique to dictionary
            
            start = int(label[1])
            end = int(label[2])
            taggedLine = wholeArticle[start:end]
            taggedLine = taggedLine.replace("\n", " ")
            taggedLine = taggedLine.replace("\t", " ")
            
            propagandaTechniques["Sentence"].append(taggedLine) # add snippet to dictionary
    f.close()

*Using the list of propaganda senteces that we've gathered, create another list, 'notPropSentences' which will contain sentences from the articles that have not been annotated as propaganda.*